In [1]:
from datasets import *
from train import *
from torchvision import datasets, transforms, models

training

In [28]:
model = models.mobilenet_v3_small(weights='DEFAULT').to('cuda')
model.classifier[0] = torch.nn.Linear(576,1280)
model.classifier[3] = torch.nn.Linear(1280,1000)
# model = models.mobilenet_v3_large()
# model.load_state_dict(torch.load("best_batch_i162153.pth")['model_state_dict'])
# train_loader, val_loader,_ = load_tiny_imagenet(64,1234)
train_loader, val_loader = load_imagenet(64,12345,resize=128)
train(model,train_loader,val_loader,'cuda',lr=0.004)

Train Epoch: 0 [0/1153050 (0%)] train loss: 6.897, lr: 0.00400000
Train Epoch: 0 [51200/1153050 (4%)] train loss: 6.095, lr: 0.00396000
Train Epoch: 0 [102400/1153050 (9%)] train loss: 4.319, lr: 0.00392040
Train Epoch: 0 [153600/1153050 (13%)] train loss: 3.848, lr: 0.00388120
Train Epoch: 0 [204800/1153050 (18%)] train loss: 3.240, lr: 0.00384238
Train Epoch: 0 [256000/1153050 (22%)] train loss: 3.666, lr: 0.00380396
Train Epoch: 0 [307200/1153050 (27%)] train loss: 3.188, lr: 0.00376592
Train Epoch: 0 [358400/1153050 (31%)] train loss: 3.299, lr: 0.00372826
Train Epoch: 0 [409600/1153050 (36%)] train loss: 2.484, lr: 0.00369098
Train Epoch: 0 [460800/1153050 (40%)] train loss: 2.728, lr: 0.00365407
Train Epoch: 0 [512000/1153050 (44%)] train loss: 2.357, lr: 0.00361753
Train Epoch: 0 [563200/1153050 (49%)] train loss: 2.715, lr: 0.00358135
Train Epoch: 0 [614400/1153050 (53%)] train loss: 2.759, lr: 0.00354554
Train Epoch: 0 [665600/1153050 (58%)] train loss: 2.125, lr: 0.00351008
T

100%|██████████| 2002/2002 [02:33<00:00, 13.03it/s]

Train Epoch: 0 [468416/1153050 (100%)] train loss: 2.951, val loss: 2.284, val acc: 0.473, top5: 0.736, lr: 0.00317446
==================== best validation accuracy ====================
epoch: 0, val accuracy: 0.47311441885151856


Train Epoch: 1 [0/1153050 (0%)] train loss: 2.027, lr: 0.00317446


testing

In [11]:
test_loader = load_imagenet(128,1234,False)
test_loader128 = load_imagenet(128,1234,False,resize=128)

In [12]:
model = models.mobilenet_v3_large(weights='DEFAULT').to('cuda')
model64 = models.mobilenet_v3_small()
model64.load_state_dict(torch.load("best_batch_i162153.pth")['model_state_dict'])
model64 = model64.to('cuda')
model.eval()
model64.eval()

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [13]:
acc64_clean = validate(model64,test_loader128,'cuda')

100%|██████████| 391/391 [04:09<00:00,  1.57it/s]


In [9]:
acc_clean = validate(model,test_loader,'cuda')
acc64_clean = validate(model64,test_loader128,'cuda')

100%|██████████| 2002/2002 [02:33<00:00, 13.06it/s]


In [10]:
acc64_clean

(0.41572156700515933, 2.7379780524855963, 0.6734781488795398)

In [8]:
print("Mobilenet V3 Large Clean (224x224)\n Top 1: {:2.2f}%\t Top 5: {:2.2f}%".format(acc_clean[0]*100,acc_clean[2]*100))
print("Mobilenet V3 Large Clean (64x64)\n Top 1: {:2.2f}%\t Top 5: {:2.2f}%".format(acc64_clean[0]*100,acc64_clean[2]*100))

Mobilenet V3 Large Clean (224x224)
 Top 1: 75.31%	 Top 5: 92.63%
Mobilenet V3 Large Clean (64x64)
 Top 1: 38.34%	 Top 5: 63.53%


In [27]:
from thop import profile
model = model = models.mobilenet_v3_large()
input = torch.randn(1, 3, 224, 224)
macs, params = profile(model, inputs=(input, ))
print("224x224:",macs/1e6,params)

model = models.mobilenet_v3_small(weights='DEFAULT')
model.classifier[0] = torch.nn.Linear(576,1280)
model.classifier[3] = torch.nn.Linear(1280,1000)
input = torch.randn(1, 3, 128, 128)
macs, params = profile(model, inputs=(input, ))
print("64x64:",macs/1e6,params)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
224x224: 22.863256 5483032.0
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container